# Summery


*   doing evaluation with scoring
*   speed measurement



In [1]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 57.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [3]:
# Define project information
PROJECT_ID = "monobrain-development"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)


In [4]:
!pip install google-cloud-storage
from google.cloud import storage
import pandas as pd
# バケット名とファイル名を指定
bucket_name = 'not_confidencial_data_for_share'
file_name = 'output_with_categories_1107.jsonl'
local_file_path = 'output_with_categories_1107.jsonl'  # ローカルの保存先

# Storageクライアントを初期化
client = storage.Client()

# バケットとBlobオブジェクトを取得
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_name)

# ファイルをローカルにダウンロード
blob.download_to_filename(local_file_path)

In [34]:
# バケット名とファイル名を指定
bucket_name = 'not_confidencial_data_for_share'
file_name = 'qa_pairs_for_rag_1108.jsonl'
local_file_path = 'qa_pairs_for_rag_1108.jsonl'  # ローカルの保存先

# Storageクライアントを初期化
client = storage.Client()

# バケットとBlobオブジェクトを取得
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_name)

# ファイルをローカルにダウンロード
blob.download_to_filename(local_file_path)

In [6]:
!pip install sentence-transformers scann numpy jsonlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 91.7 MB/s eta 0:00:00


# **BERT and ScaNN (Model 1)**

With Category Data

In [7]:
import jsonlines
import numpy as np
import random
from sentence_transformers import SentenceTransformer
import scann
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting
import time
import json

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('output_with_categories_1107.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']} Category: {pair['category']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]
categories = [pair['category'] for pair in qa_pairs]

# Initialize model for SBERT
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Step 3: Encode the documents using SBERT
question_embeddings = model_sbert.encode(documents, convert_to_numpy=True)

# Step 4: Create ScaNN index for SBERT embeddings
scann_index = scann.scann_ops.builder(question_embeddings, 10, "dot_product").tree(num_leaves=min(1000, len(questions)), num_leaves_to_search=50).score_ah(2, anisotropic_quantization_threshold=0.2).reorder(50).build()

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 5: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Encode the query using SBERT
    query_embedding = model_sbert.encode([f"Question: {query}"], convert_to_numpy=True)

    # Search the ScaNN index
    neighbors, distances = scann_index.search_batched(query_embedding)

    # Return the most similar questions
    closest_questions = [questions[i] for i in neighbors[0][:top_k]]
    return closest_questions

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。\n\n
        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    score = float(json.loads(responses.candidates[0].content.parts[0].text)['similarity'])
    return score

# Evaluate the system with 20 selected questions using Word2Vec
similarity_scores = []
#time
total_time = 0.0

# Iterate over each record to find the top 5 similar questions and calculate similarity scores + time for rag
for record in evaluation_questions:
    start_time = time.time()
    similar5data = find_similar_questions(record)
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag
    for one_data in similar5data:
        score = similarity_assessment_model(record, one_data)
        similarity_scores.append(score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
    print(f"\nThe average score of this model is: {average_similarity}")
else:
    print("\nNo similarity scores were calculated.")

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_questions)
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


The average score of this model is: 2.01

The average time taken for RAG retrieval is: 0.0399 seconds


Without Category Data

In [8]:
import jsonlines
import numpy as np
import random
from sentence_transformers import SentenceTransformer
import scann
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting
import time
import json

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('qa_pairs_for_rag_1108.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]

# Initialize model for SBERT
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Step 3: Encode the documents using SBERT
question_embeddings = model_sbert.encode(documents, convert_to_numpy=True)

# Step 4: Create ScaNN index for SBERT embeddings
scann_index = scann.scann_ops.builder(question_embeddings, 10, "dot_product").tree(num_leaves=min(1000, len(questions)), num_leaves_to_search=50).score_ah(2, anisotropic_quantization_threshold=0.2).reorder(50).build()

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 5: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Encode the query using SBERT
    query_embedding = model_sbert.encode([f"Question: {query}"], convert_to_numpy=True)

    # Search the ScaNN index
    neighbors, distances = scann_index.search_batched(query_embedding)

    # Return the most similar questions
    closest_questions = [questions[i] for i in neighbors[0][:top_k]]
    return closest_questions

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。\n\n
        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    score = float(json.loads(responses.candidates[0].content.parts[0].text)['similarity'])
    return score

# Evaluate the system with 20 selected questions using Word2Vec
similarity_scores = []
#time
total_time = 0.0

# Iterate over each record to find the top 5 similar questions and calculate similarity scores + time for rag
for record in evaluation_questions:
    start_time = time.time()
    similar5data = find_similar_questions(record)
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag
    for one_data in similar5data:
        score = similarity_assessment_model(record, one_data)
        similarity_scores.append(score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
    print(f"\nThe average score of this model is: {average_similarity}")
else:
    print("\nNo similarity scores were calculated.")

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_questions)
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")


The average score of this model is: 2.27

The average time taken for RAG retrieval is: 0.0274 seconds


# **T5 & FAISS (Model 2)**

In [9]:
!pip install transformers
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 23.8 MB/s eta 0:00:00


With Category Data

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import faiss
import jsonlines
import numpy as np
import random
import json
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('output_with_categories_1107.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']} Category: {pair['category']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]
categories = [pair['category'] for pair in qa_pairs]

# Initialize tokenizer and model for T5
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

# Step 3: Function to generate embeddings using T5
def generate_embedding(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    with torch.no_grad():
        output = model.encoder(input_ids)
    embeddings = output.last_hidden_state.mean(dim=1).squeeze().numpy()  # Average pooling
    return embeddings

# Step 4: Generate embeddings for each question
question_embeddings = [generate_embedding(doc) for doc in documents]

# Step 5: Initialize Faiss index
d = question_embeddings[0].shape[0]  # Dimensionality of embeddings
faiss_index = faiss.IndexFlatL2(d)
faiss_index.add(np.array(question_embeddings))

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 6: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Generate embedding for the query
    query_embedding = generate_embedding(f"Question: {query}")

    # Search Faiss index for nearest neighbors
    distances, indices = faiss_index.search(query_embedding.reshape(1, -1), top_k)

    # Return the most similar questions
    closest_questions = [questions[i] for i in indices[0]]
    return closest_questions

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。\n\n
        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    score = float(json.loads(responses.candidates[0].content.parts[0].text)['similarity'])
    return score

# Evaluate the system with 20 selected questions using T5 and Faiss
similarity_scores = []
#time
total_time = 0.0

# Iterate over each record to find the top 5 similar questions and calculate similarity scores
for record in evaluation_questions:
    start_time = time.time()
    similar5data = find_similar_questions(record)
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag
    for one_data in similar5data:
        score = similarity_assessment_model(record, one_data)
        similarity_scores.append(score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
    print(f"\nThe average score of this model is: {average_similarity}")
else:
    print("\nNo similarity scores were calculated.")

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_questions)
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


The average score of this model is: 1.24

The average time taken for RAG retrieval is: 0.0716 seconds


Without Category Data

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import faiss
import jsonlines
import numpy as np
import random
import json
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('qa_pairs_for_rag_1108.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]

# Initialize tokenizer and model for T5
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

# Step 3: Function to generate embeddings using T5
def generate_embedding(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    with torch.no_grad():
        output = model.encoder(input_ids)
    embeddings = output.last_hidden_state.mean(dim=1).squeeze().numpy()  # Average pooling
    return embeddings

# Step 4: Generate embeddings for each question
question_embeddings = [generate_embedding(doc) for doc in documents]

# Step 5: Initialize Faiss index
d = question_embeddings[0].shape[0]  # Dimensionality of embeddings
faiss_index = faiss.IndexFlatL2(d)
faiss_index.add(np.array(question_embeddings))

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 6: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Generate embedding for the query
    query_embedding = generate_embedding(f"Question: {query}")

    # Search Faiss index for nearest neighbors
    distances, indices = faiss_index.search(query_embedding.reshape(1, -1), top_k)

    # Return the most similar questions
    closest_questions = [questions[i] for i in indices[0]]
    return closest_questions

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。\n\n
        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    score = float(json.loads(responses.candidates[0].content.parts[0].text)['similarity'])
    return score

# Evaluate the system with 20 selected questions using T5 and Faiss
similarity_scores = []
#time
total_time = 0.0

# Iterate over each record to find the top 5 similar questions and calculate similarity scores
for record in evaluation_questions:
    start_time = time.time()
    similar5data = find_similar_questions(record)
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag
    for one_data in similar5data:
        score = similarity_assessment_model(record, one_data)
        similarity_scores.append(score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
    print(f"\nThe average score of this model is: {average_similarity}")
else:
    print("\nNo similarity scores were calculated.")

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_questions)
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")


The average score of this model is: 1.57

The average time taken for RAG retrieval is: 0.0609 seconds


# **TF-IDF + Annoy (Model 3)**

In [14]:
!pip install scikit-learn annoy jsonlines --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


With Category Data

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from annoy import AnnoyIndex
import jsonlines
import numpy as np
import random
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('output_with_categories_1107.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']} Category: {pair['category']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]
categories = [pair['category'] for pair in qa_pairs]

# Step 3: Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Convert the documents to TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(documents).toarray()  # Convert to numpy array

# Step 4: Initialize the ANNoy index
dim = tfidf_matrix.shape[1]  # Dimensionality of the vectors (number of features)
annoy_index = AnnoyIndex(dim, 'angular')  # 'angular' is cosine similarity equivalent

# Step 5: Add the TF-IDF vectors to the ANNoy index
for i, vec in enumerate(tfidf_matrix):
    annoy_index.add_item(i, vec)

# Build the index (this step can take a while depending on the number of documents)
annoy_index.build(10)  # 10 trees is a good trade-off for speed and accuracy

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 6: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Convert the query to a TF-IDF vector
    query_tfidf = vectorizer.transform([f"Question: {query}"]).toarray()

    # Get the nearest neighbors from ANNoy
    nearest_neighbors = annoy_index.get_nns_by_vector(query_tfidf[0], top_k, include_distances=False)

    # Return the most similar questions
    closest_questions = [questions[i] for i in nearest_neighbors]
    return closest_questions

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。\n\n
        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    score = float(json.loads(responses.candidates[0].content.parts[0].text)['similarity'])
    return score

# Evaluate the system with 20 selected questions using Word2Vec
similarity_scores = []
#time
total_time = 0.0

# Iterate over each record to find the top 5 similar questions and calculate similarity scores
for record in evaluation_questions:
    start_time = time.time()
    similar5data = find_similar_questions(record)
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag
    for one_data in similar5data:
        score = similarity_assessment_model(record, one_data)
        similarity_scores.append(score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
    print(f"\nThe average score of this model is: {average_similarity}")
else:
    print("\nNo similarity scores were calculated.")

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_questions)
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")


The average score of this model is: 2.3

The average time taken for RAG retrieval is: 0.0101 seconds


Without Category Data

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from annoy import AnnoyIndex
import jsonlines
import numpy as np
import random
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('qa_pairs_for_rag_1108.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]

# Step 3: Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Convert the documents to TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(documents).toarray()  # Convert to numpy array

# Step 4: Initialize the ANNoy index
dim = tfidf_matrix.shape[1]  # Dimensionality of the vectors (number of features)
annoy_index = AnnoyIndex(dim, 'angular')  # 'angular' is cosine similarity equivalent

# Step 5: Add the TF-IDF vectors to the ANNoy index
for i, vec in enumerate(tfidf_matrix):
    annoy_index.add_item(i, vec)

# Build the index (this step can take a while depending on the number of documents)
annoy_index.build(10)  # 10 trees is a good trade-off for speed and accuracy

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 6: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Convert the query to a TF-IDF vector
    query_tfidf = vectorizer.transform([f"Question: {query}"]).toarray()

    # Get the nearest neighbors from ANNoy
    nearest_neighbors = annoy_index.get_nns_by_vector(query_tfidf[0], top_k, include_distances=False)

    # Return the most similar questions
    closest_questions = [questions[i] for i in nearest_neighbors]
    return closest_questions

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。\n\n
        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    score = float(json.loads(responses.candidates[0].content.parts[0].text)['similarity'])
    return score

# Evaluate the system with 20 selected questions using Word2Vec
similarity_scores = []

#time
total_time = 0.0

# Iterate over each record to find the top 5 similar questions and calculate similarity scores
for record in evaluation_questions:
    start_time = time.time()
    similar5data = find_similar_questions(record)
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag
    for one_data in similar5data:
        score = similarity_assessment_model(record, one_data)
        similarity_scores.append(score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
    print(f"\nThe average score of this model is: {average_similarity}")
else:
    print("\nNo similarity scores were calculated.")

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_questions)
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")


The average score of this model is: 2.72

The average time taken for RAG retrieval is: 0.0083 seconds


# **Universal Sentence Encoder + HNSW (Model 4)**

In [19]:
!pip install tensorflow tensorflow-hub hnswlib numpy

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2360797 sha256=c7b8f0706e48359c0d1cff661de35a0e9bca468d2135a8947eec0e9221dfa63d
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


With Category Data

In [20]:
import jsonlines
import numpy as np
import random
import tensorflow as tf
import tensorflow_hub as hub
import hnswlib
import json
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('output_with_categories_1107.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']} Category: {pair['category']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]
categories = [pair['category'] for pair in qa_pairs]

# Load Universal Sentence Encoder model from TensorFlow Hub
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Step 3: Encode the documents using USE
question_embeddings = use_model(documents).numpy()

# Step 4: Create HNSW index
dim = question_embeddings.shape[1]
p = hnswlib.Index(space='cosine', dim=dim)
p.init_index(max_elements=len(questions), ef_construction=200, M=16)
p.add_items(question_embeddings)
p.set_ef(50)  # ef should be set to a value between 10 and 100 for best performance

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 5: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Encode the query using USE
    query_embedding = use_model([f"Question: {query}"]).numpy()

    # Search the HNSW index
    labels, distances = p.knn_query(query_embedding, k=top_k)

    # Return the most similar questions
    closest_questions = [questions[i] for i in labels[0]]
    return closest_questions

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。\n\n
        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    try:
        response_content = responses.candidates[0].content.parts[0].text
        score = float(json.loads(response_content)['similarity'])
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error decoding JSON response: {e}")
        score = 0.0
    return score

# Evaluate the system with 20 selected questions using USE and HNSW
similarity_scores = []

#time
total_time = 0.0

# Iterate over each record to find the top 5 similar questions and calculate similarity scores
for record in evaluation_questions:
    start_time = time.time()
    similar5data = find_similar_questions(record)
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag
    for one_data in similar5data:
        score = similarity_assessment_model(record, one_data)
        similarity_scores.append(score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
    print(f"\nThe average score of this model is: {average_similarity}")
else:
    print("\nNo similarity scores were calculated.")

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_questions)
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")


The average score of this model is: 0.68

The average time taken for RAG retrieval is: 0.0043 seconds


Without Category Data

In [21]:
import jsonlines
import numpy as np
import random
import tensorflow as tf
import tensorflow_hub as hub
import hnswlib
import json
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('qa_pairs_for_rag_1108.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']} " for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]

# Load Universal Sentence Encoder model from TensorFlow Hub
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Step 3: Encode the documents using USE
question_embeddings = use_model(documents).numpy()

# Step 4: Create HNSW index
dim = question_embeddings.shape[1]
p = hnswlib.Index(space='cosine', dim=dim)
p.init_index(max_elements=len(questions), ef_construction=200, M=16)
p.add_items(question_embeddings)
p.set_ef(50)  # ef should be set to a value between 10 and 100 for best performance

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 5: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Encode the query using USE
    query_embedding = use_model([f"Question: {query}"]).numpy()

    # Search the HNSW index
    labels, distances = p.knn_query(query_embedding, k=top_k)

    # Return the most similar questions
    closest_questions = [questions[i] for i in labels[0]]
    return closest_questions

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。\n\n
        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    try:
        response_content = responses.candidates[0].content.parts[0].text
        score = float(json.loads(response_content)['similarity'])
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error decoding JSON response: {e}")
        score = 0.0
    return score

# Evaluate the system with 20 selected questions using USE and HNSW
similarity_scores = []

#time
total_time = 0.0

# Iterate over each record to find the top 5 similar questions and calculate similarity scores
for record in evaluation_questions:
    start_time = time.time()
    similar5data = find_similar_questions(record)
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag
    for one_data in similar5data:
        score = similarity_assessment_model(record, one_data)
        similarity_scores.append(score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
    print(f"\nThe average score of this model is: {average_similarity}")
else:
    print("\nNo similarity scores were calculated.")

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_questions)
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")


The average score of this model is: 0.76

The average time taken for RAG retrieval is: 0.0041 seconds


# **Word2Vec Implementation (Model 5)**

In [22]:
!pip install jsonlines
!pip install nltk
!pip install gensim
!pip install janome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 77.3 MB/s eta 0:00:00


With Category Data

In [23]:
import jsonlines
import numpy as np
import random
import json
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from janome.tokenizer import Tokenizer  # For Japanese tokenization
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

# Initialize Janome tokenizer for Japanese
tokenizer = Tokenizer()

# Function to tokenize Japanese text
def tokenize_japanese(text):
    return [token.surface for token in tokenizer.tokenize(text)]

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('output_with_categories_1107.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']} Category: {pair['category']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]
categories = [pair['category'] for pair in qa_pairs]

# Tokenize data using Janome for Japanese text
tokenized_data = []
for item in qa_pairs:
    question_tokens = tokenize_japanese(item['question'])
    answer_tokens = tokenize_japanese(item['answer'])
    category_tokens = tokenize_japanese(item['category'])
    # Flatten question, answer, and category into the same list for Word2Vec training
    tokenized_data.append(question_tokens + answer_tokens + category_tokens)

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

# Create a vectorized representation of questions without averaging word vectors
def get_combined_vector(tokens):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.vstack(vectors)  # Stack vectors vertically
    else:
        return None

question_vectors = []
for item in qa_pairs:
    question_tokens = tokenize_japanese(item['question'])
    question_vec = get_combined_vector(question_tokens)
    question_vectors.append(question_vec)

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Function to find the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    query_tokens = tokenize_japanese(query)
    query_vec = get_combined_vector(query_tokens)

    if query_vec is not None:
        valid_vectors = [(i, vec) for i, vec in enumerate(question_vectors) if vec is not None]
        similarities = []
        for i, vec in valid_vectors:
            similarity = cosine_similarity(query_vec, vec).mean()  # Calculate similarity between stacked vectors
            similarities.append((i, similarity))
        similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
        most_similar_indices = [i for i, _ in similarities[:top_k]]
        closest_questions = [questions[i] for i in most_similar_indices]
        return closest_questions
    else:
        return []

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。\n\n
        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    score = float(json.loads(responses.candidates[0].content.parts[0].text)['similarity'])
    return score

# Evaluate the system with 20 selected questions using Word2Vec
similarity_scores = []
#time
total_time = 0.0

# Iterate over each record to find the top 5 similar questions and calculate similarity scores
for record in evaluation_questions:
    start_time = time.time()
    similar5data = find_similar_questions(record)
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag
    for one_data in similar5data:
        score = similarity_assessment_model(record, one_data)
        similarity_scores.append(score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
    print(f"\nThe average score of this model is: {average_similarity}")
else:
    print("\nNo similarity scores were calculated.")

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_questions)
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")



The average score of this model is: 0.89

The average time taken for RAG retrieval is: 1.4757 seconds


Without Category Data

In [24]:
import jsonlines
import numpy as np
import random
import json
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from janome.tokenizer import Tokenizer  # For Japanese tokenization
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

# Initialize Janome tokenizer for Japanese
tokenizer = Tokenizer()

# Function to tokenize Japanese text
def tokenize_japanese(text):
    return [token.surface for token in tokenizer.tokenize(text)]

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('qa_pairs_for_rag_1108.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]

# Tokenize data using Janome for Japanese text
tokenized_data = []
for item in qa_pairs:
    question_tokens = tokenize_japanese(item['question'])
    answer_tokens = tokenize_japanese(item['answer'])
    # Flatten question, answer, and category into the same list for Word2Vec training
    tokenized_data.append(question_tokens + answer_tokens)

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

# Create a vectorized representation of questions without averaging word vectors
def get_combined_vector(tokens):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.vstack(vectors)  # Stack vectors vertically
    else:
        return None

question_vectors = []
for item in qa_pairs:
    question_tokens = tokenize_japanese(item['question'])
    question_vec = get_combined_vector(question_tokens)
    question_vectors.append(question_vec)

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Function to find the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    query_tokens = tokenize_japanese(query)
    query_vec = get_combined_vector(query_tokens)

    if query_vec is not None:
        valid_vectors = [(i, vec) for i, vec in enumerate(question_vectors) if vec is not None]
        similarities = []
        for i, vec in valid_vectors:
            similarity = cosine_similarity(query_vec, vec).mean()  # Calculate similarity between stacked vectors
            similarities.append((i, similarity))
        similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
        most_similar_indices = [i for i, _ in similarities[:top_k]]
        closest_questions = [questions[i] for i in most_similar_indices]
        return closest_questions
    else:
        return []

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。\n\n
        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    score = float(json.loads(responses.candidates[0].content.parts[0].text)['similarity'])
    return score

# Evaluate the system with 20 selected questions using Word2Vec
similarity_scores = []
#time
total_time = 0.0

# Iterate over each record to find the top 5 similar questions and calculate similarity scores
for record in evaluation_questions:
    start_time = time.time()
    similar5data = find_similar_questions(record)
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag
    for one_data in similar5data:
        score = similarity_assessment_model(record, one_data)
        similarity_scores.append(score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
    print(f"\nThe average score of this model is: {average_similarity}")
else:
    print("\nNo similarity scores were calculated.")

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_questions)
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")



The average score of this model is: 1.01

The average time taken for RAG retrieval is: 1.4214 seconds


# **Agent Builder**

In [25]:
!pip install google-cloud-discoveryengine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 24.3 MB/s eta 0:00:00


In [26]:
from typing import List

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine

# TODO(developer): Uncomment these variables before running the sample.
# project_id = "YOUR_PROJECT_ID"
# location = "YOUR_LOCATION"          # Values: "global", "us", "eu"
# engine_id = "YOUR_APP_ID"
# search_query = "YOUR_SEARCH_QUERY"


def search_sample(
    project_id: str,
    location: str,
    engine_id: str,
    search_query: str,
) -> List[discoveryengine.SearchResponse]:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search app serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    # Optional - only supported for unstructured data: Configuration options for search.
    # Refer to the `ContentSearchSpec` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble="YOUR_CUSTOM_PROMPT"
            ),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="stable",
            ),
        ),
    )

    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    #print(response)

    return response


In [27]:
print(search_sample(
project_id = "monobrain-development",
location = "global",                    # Values: "global", "us", "eu"
engine_id = "sample-category_1732854393042",
search_query = "今日は取り組めませんでした"
))
type(search_sample(
project_id = "monobrain-development",
location = "global",                    # Values: "global", "us", "eu"
engine_id = "sample-category_1732854393042",
search_query = "今日は取り組めませんでした"
))

SearchPager<results {
  id: "e36874abcb88c00019bbf57dc8a4ae00"
  document {
    struct_data {
      fields {
        key: "question"
        value {
          string_value: "今日は何も取り組めませんでした。"
        }
      }
      fields {
        key: "id"
        value {
          string_value: "qa_433"
        }
      }
      fields {
        key: "category"
        value {
          string_value: "今日は取り組めず、連絡。\n"
        }
      }
      fields {
        key: "answer"
        value {
          string_value: "ご報告いただきありがとうございます♪"
        }
      }
    }
    name: "projects/954273464710/locations/global/collections/default_collection/dataStores/category_1732854489356/branches/0/documents/e36874abcb88c00019bbf57dc8a4ae00"
    id: "e36874abcb88c00019bbf57dc8a4ae00"
    derived_struct_data {
      fields {
        key: "snippets"
        value {
          list_value {
            values {
              struct_value {
                fields {
                  key: "snippet"
                  value {
   

google.cloud.discoveryengine_v1.services.search_service.pagers.SearchPager

In [31]:
from google.cloud.discoveryengine_v1.services.search_service.pagers import SearchPager

from google.cloud.discoveryengine_v1.services.search_service.pagers import SearchPager


# Transform the response into desired format
def transform_search_response(response) -> list:
    similar_chats = []
    for result in response.results:
        data = dict(result.document.struct_data)
        chat = {
            'id': data.get('id', ''),
            'question': data.get('question', ''),
            'answer': data.get('answer', ''),
            'category': data.get('category', ''),
        }
        similar_chats.append(chat)
    return similar_chats

# Fetch data from multiple pages
def fetch_all_data(project_id, location, engine_id, num_pages=5, page_size=20):
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    client = discoveryengine.SearchServiceClient(client_options=client_options)
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    all_data = []
    for page in range(num_pages):
        request = discoveryengine.SearchRequest(
            serving_config=serving_config,
            query="",  # Broad query to fetch data
            page_size=page_size,
            offset=page * page_size
        )
        response = client.search(request)
        all_data.extend(transform_search_response(response))

    return all_data

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。

        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    score = float(json.loads(responses.candidates[0].content.parts[0].text)['similarity'])
    return score

# Step 1: Fetch data from multiple pages
all_data = fetch_all_data(
    project_id="monobrain-development",
    location="global",
    engine_id="sample-category_1732854393042",
    num_pages=5,
    page_size=20
)

# Step 2: Randomly select 20 entries for evaluation
evaluation_data = random.sample(all_data, min(20, len(all_data)))

# Step 3: Evaluate each query and find 5 similar questions
similarity_scores = []
total_time = 0.0

for idx, data in enumerate(evaluation_data, start=1):
    query = data['question']

    # Query the engine for similar questions
    start_time = time.time()
    similar_response = search_sample(
        project_id="monobrain-development",
        location="global",
        engine_id="sample-category_1732854393042",
        search_query=query
    )
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag

    # Transform the response to extract similar chats
    similar_chats = transform_search_response(similar_response)[:5]

    for i, chat in enumerate(similar_chats, start=1):
        similarity_score = similarity_assessment_model(query, chat['question'])
        similarity_scores.append(similarity_score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
else:
    average_similarity = 0.0

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_data)

# Output the results
print(f"\nThe average score of this model is: {average_similarity:.2f}")
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")



The average score of this model is: 2.88

The average time taken for RAG retrieval is: 1.0700 seconds


Without Category Data

In [36]:
print(search_sample(
project_id = "monobrain-development",
location = "global",                    # Values: "global", "us", "eu"
engine_id = "samplequery_1732851004864",
search_query = "今日は取り組めませんでした"
))


SearchPager<results {
  id: "2f2fda00a42f572696cc306f99c94c53"
  document {
    struct_data {
      fields {
        key: "question"
        value {
          string_value: "谷津さんお久しぶりです1週間程体調を崩してしまい、あまり進められませんでした。取り組みもご報告できておらず、すみませんでした。本日からまた頑張っていきます！取り組んだこと10/4・2-2 動画視聴10/5・2-2 実践課題実施10/6・2-2 実践課題提出・2-3 動画視聴10/8・勉強会・2-3 実践課題実施、提出10/14・2-3 実践課題実施、再提出・3-1 動画視聴、実践課題提出・3-2 動画視聴、実践課題提出・3-3 動画視聴、実践課題実施"
        }
      }
      fields {
        key: "id"
        value {
          string_value: "qa_1341"
        }
      }
      fields {
        key: "answer"
        value {
          string_value: "ご報告ありがとうございます！体調不良の中でも、コツコツ進めてくださり素晴らしいです🥲体調第一で、引き続きご自身の良いペースで無理はしすぎず進めていきましょう😊お取り組みお疲れ様でした👏✨"
        }
      }
    }
    name: "projects/954273464710/locations/global/collections/default_collection/dataStores/query_1732851097294/branches/0/documents/2f2fda00a42f572696cc306f99c94c53"
    id: "2f2fda00a42f572696cc306f99c94c53"
    derived_struct_data {
      fields {
        key: "snippets"
      

In [37]:
from google.cloud.discoveryengine_v1.services.search_service.pagers import SearchPager

from google.cloud.discoveryengine_v1.services.search_service.pagers import SearchPager


# Transform the response into desired format
def transform_search_response(response) -> list:
    similar_chats = []
    for result in response.results:
        data = dict(result.document.struct_data)
        chat = {
            'id': data.get('id', ''),
            'question': data.get('question', ''),
            'answer': data.get('answer', ''),
            #'category': data.get('category', ''),
        }
        similar_chats.append(chat)
    return similar_chats

# Fetch data from multiple pages
def fetch_all_data(project_id, location, engine_id, num_pages=5, page_size=20):
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    client = discoveryengine.SearchServiceClient(client_options=client_options)
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    all_data = []
    for page in range(num_pages):
        request = discoveryengine.SearchRequest(
            serving_config=serving_config,
            query="",  # Broad query to fetch data
            page_size=page_size,
            offset=page * page_size
        )
        response = client.search(request)
        all_data.extend(transform_search_response(response))

    return all_data

# Function to assess similarity between two queries using Vertex AI
def similarity_assessment_model(query1, query2):
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "response_mime_type": "application/json"
    }

    vertexai.init(project="monobrain-development", location="us-central1")
    model = GenerativeModel("gemini-1.5-flash-001")
    responses = model.generate_content([
        """あなたは日本語の小説の編集者をしており、文章のスペシャリストです。
        以下の文章を比べて類似度を数値5段階で出力してください。

        文章の大まかな内容と細かい内容、文章表現全てが類似している場合は : 5
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が類似しているものは : 4
        文章の大まかな内容は類似しているが、細かい内容が違っており、文章表現が異なっているものは : 3
        文章の大まかな内容は異なっているが、文内で使われている単語が似通っている場合 : 2
        文書の内容は間違っているが、文章表現や文章の雰囲気が似ている場合 : 1
        文章の内容が違っており、文章表現や単語も異なっている場合 : 0

        jsonの形式は{"similarity":3,"comment":"ここになぜそのような評価にしたかのコメント"}

        文章1 :""" + query1 + "\n 文章2 : " + query2],
        generation_config=generation_config,
        stream=False,
    )

    score = float(json.loads(responses.candidates[0].content.parts[0].text)['similarity'])
    return score

# Step 1: Fetch data from multiple pages
all_data = fetch_all_data(
    project_id="monobrain-development",
    location="global",
    engine_id="samplequery_1732851004864",
    num_pages=5,
    page_size=20
)

# Step 2: Randomly select 20 entries for evaluation
evaluation_data = random.sample(all_data, min(20, len(all_data)))

# Step 3: Evaluate each query and find 5 similar questions
similarity_scores = []
total_time = 0.0

for idx, data in enumerate(evaluation_data, start=1):
    query = data['question']

    # Query the engine for similar questions
    start_time = time.time()
    similar_response = search_sample(
        project_id="monobrain-development",
        location="global",
        engine_id="samplequery_1732851004864",
        search_query=query
    )
    end_time = time.time()
    time_for_using_rag = end_time - start_time
    total_time += time_for_using_rag

    # Transform the response to extract similar chats
    similar_chats = transform_search_response(similar_response)[:5]

    for i, chat in enumerate(similar_chats, start=1):
        similarity_score = similarity_assessment_model(query, chat['question'])
        similarity_scores.append(similarity_score)

# Calculate the average similarity score
if similarity_scores:
    average_similarity = np.mean(similarity_scores)
else:
    average_similarity = 0.0

# Calculate the average time taken for RAG retrieval
average_time = total_time / len(evaluation_data)

# Output the results
print(f"\nThe average score of this model is: {average_similarity:.2f}")
print(f"\nThe average time taken for RAG retrieval is: {average_time:.4f} seconds")



The average score of this model is: 2.05

The average time taken for RAG retrieval is: 0.9076 seconds
